In [1]:
# poetry run pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import json
import re
from tqdm import tqdm

c:\Users\pschm\OneDrive\eng\projetos\linkedin_jobs_analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# MODEL_NAME = "Qwen/Qwen1.5-1.8B-Chat"  # Smaller and instruction-tuned
MODEL_NAME = 'Qwen/Qwen2.5-7B'
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [19]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    device_map="auto"
)

c:\Users\pschm\OneDrive\eng\projetos\linkedin_jobs_analysis\.venv\Lib\site-packages\accelerate\utils\modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.42s/it]
Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [20]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device=DEVICE,
)

Device set to use cpu


In [21]:
def extract_requirements_qwen(job_description: str) -> dict:
    """Optimized extraction function with better prompt engineering"""

    system_prompt = """
    You are an expert at extracting required skills from job descriptions. 
    Given the job description below from a data related position, identify all the technical and professional skills mentioned. 
    Return the skills as a JSON object with the key "skills" containing a list of strings written in pt-br.
    Do not include irrelevant information or explanations and only include skills that you consider essential to the data area.
    

    Job Description:
    """

    # Construct messages
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": job_description[:2000]}
    ]

    # Generate prompt
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Generate output
    outputs = pipe(
        prompt,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False
    )

    # Extract and clean output
    raw_output = outputs[0]['generated_text'].strip()
    json_str = raw_output.replace("```json", "").replace("```", "").strip()

    try:
        result = json.loads(json_str)
        if "skills" in result and isinstance(result["skills"], list):
            return result
        else:
            return {"error": "Invalid JSON structure"}
    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON output"}

In [22]:
job_descriptions = pd.read_csv('../data/raw/jobs_data.csv')['job_description']
job_descriptions

0    Descrição\nEstamos em busca de um(a) estagiári...
1    Lugar de gente inovadora que faz acontecer!\nS...
2    Há mais de 90 anos, o Banco Bmg vem construind...
3    Somos uma indústria brasileira com atuação glo...
4    About The Team\nSobre a Shô\nAqui na Shô, acre...
5    Que tal construir a REDE com a gente? 🚀\nHá 26...
6    Descrição\nO estagiário(a) de Sistemas da Info...
7    Vem construir no Itaú_\nComo maior banco priva...
8    A NEON, ciente de sua responsabilidade social ...
9    Somos uma empresa brasileira de varejo reconhe...
Name: job_description, dtype: object

In [23]:
for description in job_descriptions:
    result = extract_requirements_qwen(description)
    print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "error": "Failed to parse JSON output"
}
{
  "error": "Failed to parse JSON output"
}
{
  "error": "Failed to parse JSON output"
}


KeyboardInterrupt: 

---

In [48]:
skill_list = [
    # Programming Languages
    "Python", "R", "SQL", "Julia", "Scala", "Java", "C++", "MATLAB", "SAS", 
    "JavaScript", "TypeScript", "Bash", "Go", "Rust",

    # Databases & Data Storage
    "PostgreSQL", "MySQL", "SQLite", "SQL Server", "Oracle Database", "MongoDB", 
    "Cassandra", "Redis", "Elasticsearch", "InfluxDB", "Neo4j", "DynamoDB", "BigQuery", 
    "Snowflake", "Redshift", "ClickHouse", "DuckDB", "MariaDB", "Firebase Realtime Database",

    # Data Processing & ETL
    "Pandas", "Polars", "Dask", "Modin", "Vaex", "Koalas", "PySpark", "Apache Spark", 
    "Apache Flink", "Apache Beam", "Airflow", "Luigi", "Prefect", "Kubernetes", 
    "Docker", "Kafka", "RabbitMQ", "Celery", "DBT (Data Build Tool)", "Great Expectations", 
    "Hadoop", "Databricks", "Azure Data Factory", "AWS Glue",

    # Business Intelligence & Data Visualization
    "Power BI", "Tableau", "Looker", "Google Data Studio", "Qlik Sense", "Metabase", 
    "Superset", "Dash", "Streamlit", "Plotly", "Matplotlib", "Seaborn", "Bokeh", 
    "Altair", "ggplot2", "Grafana", "D3.js", "ECharts", 

    # Machine Learning & AI
    "Scikit-learn", "TensorFlow", "Keras", "PyTorch", "XGBoost", "LightGBM", "CatBoost", 
    "Hugging Face Transformers", "OpenCV", "DeepFace", "Numpy", "Scipy", "Statsmodels", 
    "Fastai", "EvidentlyAI", "Optuna", "Hyperopt", "Ray Tune", "MLflow", "Weights & Biases", 
    "AutoML", "H2O.ai", "PyCaret", "TPOT", "Turi Create",

    # Natural Language Processing (NLP)
    "NLTK", "spaCy", "Gensim", "TextBlob", "BERT", "GPT", "Word2Vec", "FastText", 
    "SentenceTransformers", "LlamaIndex", "Haystack", "Rasa", "LangChain", "Llama",

    # Computer Vision
    "OpenCV", "Detectron2", "YOLO", "MMDetection", "DeepFace", "Mediapipe", "Dlib", 
    "TensorFlow Object Detection API", "Albumentations", "TorchVision",

    # Big Data & Distributed Computing
    "Apache Spark", "Dask", "Ray", "Apache Flink", "Hadoop", "Flink", "Google BigQuery", 
    "Presto", "Trino", "ClickHouse", "Hive", "HBase",

    # MLOps & Model Deployment
    "MLflow", "TensorFlow Serving", "TorchServe", "Kubeflow", "Seldon", "BentoML", 
    "FastAPI", "Flask", "Streamlit", "Triton Inference Server", "ONNX", "AWS SageMaker", 
    "Azure ML", "Google Vertex AI", "Ray Serve",

    # Cloud Platforms
    "AWS", "Azure", "Google Cloud Platform", "IBM Cloud", "Oracle Cloud", 
    "Snowflake", "Databricks", "Redshift", "BigQuery", "DataBricks", "Terraform", 
    "Kubernetes", "Docker",

    # Statistical & Mathematical Foundations
    "Linear Algebra", "Calculus", "Probability", "Bayesian Statistics", "Hypothesis Testing", 
    "Descriptive Statistics", "Inferential Statistics", "Optimization", "Time Series Analysis", 
    "Markov Chains", "Graph Theory", "Stochastic Processes",

    # Feature Engineering
    "Feature Selection", "Feature Scaling", "Dimensionality Reduction", "PCA", "t-SNE", 
    "UMAP", "One-Hot Encoding", "Label Encoding", "Ordinal Encoding", "Target Encoding", 
    "Feature Crossing", "Polynomial Features",

    # Model Explainability & Fairness
    "SHAP", "LIME", "Eli5", "Fairlearn", "Aequitas", "Facets", "Responsible AI Toolkit", 
    "What-If Tool",

    # Time Series Analysis
    "ARIMA", "SARIMA", "Prophet", "LSTM", "Holt-Winters", "VAR", "TBATS", "Kats", "GluonTS",

    # Graph Data Science
    "NetworkX", "Neo4j", "GraphFrames", "GraphX", "DGL (Deep Graph Library)", "PyG (PyTorch Geometric)", 

    # Reinforcement Learning
    "Stable-Baselines3", "RLlib", "Gym", "Mujoco", "PettingZoo", "Acme",

    # Data Governance & Privacy
    "GDPR Compliance", "CCPA Compliance", "Data Masking", "Differential Privacy", "Anonymization", 
    "Data Lineage", "Data Cataloging", "Data Security", "Data Contracts", "Data Quality Management",

    # Data Engineering
    "ETL Pipelines", "Data Warehousing", "Data Lakes", "Delta Lake", "Lakehouse Architecture", 
    "OLAP vs OLTP", "CDC (Change Data Capture)", "Data Orchestration", "Message Queues (Kafka, RabbitMQ)", 
    "Reverse ETL",

    # Experimentation & A/B Testing
    "A/B Testing", "Multivariate Testing", "Bandit Algorithms", "Bayesian Optimization", 
    "Causal Inference", "DoWhy",

    # Optimization & Operations Research
    "Linear Programming", "Mixed-Integer Programming", "Convex Optimization", "Simulated Annealing", 
    "Genetic Algorithms", "Constraint Programming",

    # Data Ethics & Bias
    "Algorithmic Fairness", "Bias Detection", "Interpretability", "Explainable AI (XAI)", "Fair AI",

    # Miscellaneous
    "Regex", "Web Scraping", "BeautifulSoup", "Scrapy", "Data Cleaning", "Data Transformation", 
    "Data Streaming", "Feature Store", "Real-Time Analytics", "DataOps", "Observability",

    # Portuguese Translations (Extra Entries)
    "Aprendizado de Máquina", "Engenharia de Dados", "Análise de Dados", "Processamento de Linguagem Natural", 
    "Ciência de Dados", "Visualização de Dados", "Estatística", "Otimização", "Big Data", 
    "Governança de Dados", "Orquestração de Dados", "Explicabilidade de Modelos", "Séries Temporais",

    # Linguagens de programação
    "Python", "R", "SQL", "Julia", "Scala", "Java", "C++", "MATLAB", "SAS",
    "JavaScript", "TypeScript", "Bash", "Go", "Rust",

    # Bancos de dados e armazenamento de dados
    "PostgreSQL", "MySQL", "SQLite", "SQL Server", "Oracle Database", "MongoDB",
    "Cassandra", "Redis", "Elasticsearch", "InfluxDB", "Neo4j", "DynamoDB", "BigQuery",
    "Snowflake", "Redshift", "ClickHouse", "DuckDB", "MariaDB", "Firebase Realtime Database",

    # Processamento de dados e ETL
    "Excel", "Pandas", "Polars", "Dask", "Modin", "Vaex", "Koalas", "PySpark", "Apache Spark",
    "Apache Flink", "Apache Beam", "Airflow", "Luigi", "Prefect", "Kubernetes",
    "Docker", "Kafka", "RabbitMQ", "Celery", "DBT (Ferramenta de construção de dados)", "Great Expectations",
    "Hadoop", "Databricks", "Azure Data Factory", "AWS Glue",

    # Inteligência de negócios e visualização de dados
    "Power BI", "Tableau", "Looker", "Google Data Studio", "Qlik Sense", "Metabase",
    "Superset", "Dash", "Streamlit", "Plotly", "Matplotlib", "Seaborn", "Bokeh",
    "Altair", "ggplot2", "Grafana", "D3.js", "ECharts", "PowerPoint", "Word"

    # Aprendizado de máquina e IA
    "Scikit-learn", "TensorFlow", "Keras", "PyTorch", "XGBoost", "LightGBM", "CatBoost",
    "Transformadores de rostos abraçados", "OpenCV", "DeepFace", "Numpy", "Scipy", "Statsmodels",
    "Fastai", "EvidentlyAI", "Optuna", "Hyperopt", "Ray Tune", "MLflow", "Pesos e vieses",
    "AutoML", "H2O.ai", "PyCaret", "TPOT", "Turi Create",

    # Processamento de linguagem natural (PLN)
    "NLTK", "spaCy", "Gensim", "TextBlob", "BERT", "GPT", "Word2Vec", "FastText",
    "Transformadores de frases", "LlamaIndex", "Haystack", "Rasa", "LangChain", "Llama",

    # Visão computacional
    "OpenCV", "Detectron2", "YOLO", "MMDetection", "DeepFace", "Mediapipe", "Dlib",
    "API de detecção de objetos TensorFlow", "Albumentações", "TorchVision",

    # Big Data e computação distribuída
    "Apache Spark", "Dask", "Ray", "Apache Flink", "Hadoop", "Flink", "Google BigQuery",
    "Presto", "Trino", "ClickHouse", "Hive", "HBase",

    # MLOps e implantação de modelo
    "MLflow", "TensorFlow Serving", "TorchServe", "Kubeflow", "Seldon", "BentoML",
    "FastAPI", "Flask", "Streamlit", "Triton Inference Server", "ONNX", "AWS SageMaker",
    "Azure ML", "Google Vertex AI", "Ray Serve",

    # Plataformas de nuvem
    "AWS", "Azure", "Google Cloud Platform", "IBM Cloud", "Oracle Cloud",
    "Snowflake", "Databricks", "Redshift", "BigQuery", "DataBricks", "Terraform",
    "Kubernetes", "Docker",

    # Fundamentos estatísticos e matemáticos
    "Álgebra linear", "Cálculo", "Probabilidade", "Estatística bayesiana", "Teste de hipóteses",
    "Estatística descritiva", "Estatística inferencial", "Otimização", "Análise de séries temporais",
    "Cadeias de Markov", "Teoria dos grafos", "Processos estocásticos",

    # Engenharia de recursos
    "Seleção de recursos", "Escalonamento de recursos", "Redução de dimensionalidade", "PCA", "t-SNE",
    "UMAP", "Codificação One-Hot", "Codificação de rótulos", "Codificação ordinal", "Codificação de destino",
    "Cruzamento de recursos", "Recursos polinomiais",

    # Explicabilidade e imparcialidade do modelo
    "SHAP", "LIME", "Eli5", "Fairlearn", "Aequitas", "Facetas", "Kit de ferramentas de IA responsável",
    "Ferramenta What-If",

    # Análise de séries temporais
    "ARIMA", "SARIMA", "Prophet", "LSTM", "Holt-Winters", "VAR", "TBATS", "Kats", "GluonTS",

    # Ciência de dados de gráfico
    "NetworkX", "Neo4j", "GraphFrames", "GraphX", "DGL (Deep Graph Library)", "PyG (PyTorch Geometric)",

    # Aprendizado por reforço
    "Stable-Baselines3", "RLlib", "Gym", "Mujoco", "PettingZoo", "Acme",

    # Governança e privacidade de dados
    "Conformidade com GDPR", "Conformidade com CCPA", "Mascaramento de dados", "Privacidade diferencial", "Anonimização",
    "Linhagem de dados", "Catalogação de dados", "Segurança de dados", "Contratos de dados", "Gerenciamento de qualidade de dados",

    # Engenharia de dados
    "Pipelines ETL", "Dados Warehousing", "Data Lakes", "Delta Lake", "Lakehouse Architecture",
    "OLAP vs OLTP", "CDC (Change Data Capture)", "Data Orchestration", "Message Queues (Kafka, RabbitMQ)",
    "Reverse ETL",

    # Experimentação e Teste A/B
    "Teste A/B", "Teste Multivariado", "Algoritmos Bandit", "Otimização Bayesiana",
    "Inferência Causal", "DoWhy",

    # Otimização e Pesquisa Operacional
    "Programação Linear", "Programação Inteira Mista", "Otimização Convexa", "Simulated Annealing",
    "Algoritmos Genéticos", "Programação de Restrições",

    # Ética e Viés de Dados
    "Justiça Algorítmica", "Detecção de Viés", "Interpretabilidade", "IA Explicável (XAI)", "IA Justa",

    # Diversos
    "Regex", "Web Scraping", "BeautifulSoup", "Scrapy", "Limpeza de dados", "Transformação de dados", 
    "Streaming de dados", "Feature Store", "Análise em tempo real", "DataOps", "Observabilidade",
]
skill_list = list(set(skill_list))
skill_list = [x.lower() for x in skill_list]

In [62]:
skill_list = [
    # Programming Languages / Tools
    "Python",
    "R",
    "SQL",
    "Excel",

    # Machine Learning & Artificial Intelligence
    "Machine Learning", "Aprendizagem de Máquina",
    "Deep Learning", "Aprendizagem Profunda",
    "Reinforcement Learning", "Aprendizagem por Reforço",
    "Artificial Intelligence", "Inteligência Artificial",
    "Algorithm Development", "Desenvolvimento de Algoritmos",

    # Data Analysis & Statistics
    "Data Analysis", "Análise de Dados",
    "Statistical Analysis", "Análise Estatística",
    "Statistics", "Estatística",
    "Predictive Analytics", "Análise Preditiva",
    "Prescriptive Analytics", "Análise Prescritiva",
    "Statistical Modeling", "Modelagem Estatística",
    
    # Data Science & Engineering
    "Data Science", "Ciência de Dados",
    "Data Engineering", "Engenharia de Dados",
    "Data Architecture", "Arquitetura de Dados",
    "Data Strategy", "Estratégia de Dados",
    
    # Data Visualization & Storytelling
    "Data Visualization", "Visualização de Dados",
    "Data Storytelling", "Contação de Histórias com Dados",
    "Business Intelligence", "Inteligência de Negócios",
    "Business Analytics", "Análise de Negócios",
    
    # Big Data & Cloud
    "Big Data", "Big Data",
    "Data Warehousing", "Armazenamento de Dados",
    "Big Data Engineering", "Engenharia de Big Data",
    "Big Data Analytics", "Análise de Big Data",
    "Cloud Computing", "Computação em Nuvem",
    "Cloud Data Services", "Serviços de Dados em Nuvem",
    
    # Data Processing & Tools
    "ETL",
    "Data Pipelines", "Fluxos de Dados",
    "Data Integration", "Integração de Dados",
    "Data Cleaning", "Limpeza de Dados",
    "Data Wrangling", "Manipulação de Dados",
    "Data Quality", "Qualidade dos Dados",
    "Data Governance", "Governança de Dados",
    "Data Platforms", "Plataformas de Dados",
    
    # Databases & Query Optimization
    "NoSQL",
    "SQL Tuning", "Otimização de SQL",
    "Query Optimization", "Otimização de Consultas",
    
    # Big Data Frameworks
    "Hadoop",
    "Spark",
    
    # Specialized Tools & Software
    "Tableau",
    "Power BI",
    "SAS",
    "Stata",
    "Matlab",
    "Docker",
    "Kubernetes",
    
    # Advanced Data Concepts
    "Feature Engineering", "Engenharia de Atributos",
    "Time Series Analysis", "Análise de Séries Temporais",
    "A/B Testing", "Testes A/B",
    "Data Mining Techniques", "Técnicas de Mineração de Dados",
    "Data Mining", "Mineração de Dados",
    "Natural Language Processing", "Processamento de Linguagem Natural",
    "Computer Vision", "Visão Computacional",
    "Information Retrieval", "Recuperação de Informação",
    
    # Security & Ethics
    "Data Security", "Segurança de Dados",
    "Data Privacy", "Privacidade de Dados",
    "Data Ethics", "Ética de Dados"
]
skill_list = list(set(skill_list))
# skill_list = [x.lower() for x in skill_list]

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher
from thefuzz import process
import re

sample_desc = """O estagiário(a) de Sistemas da Informação será responsável por prestar suporte aos usuários do ERP Datasul, atuando diretamente na resolução de problemas e na abertura e acompanhamento de chamados com a empresa fornecedora do software. O profissional também participará de treinamentos, reuniões de equipe.
Responsabilidades e atribuições
Prestar suporte técnico e funcional aos usuários do ERP Datasul.
Auxiliar na resolução de problemas relacionados ao sistema, garantindo o bom funcionamento e a continuidade das operações.
Realizar a abertura e acompanhamento de chamados técnicos junto à empresa fornecedora do software, garantindo que as solicitações sejam tratadas de maneira eficiente.
Participar de treinamentos contínuos para atualização sobre novas funcionalidades e melhorias no sistema.
Colaborar com a equipe nas reuniões para discutir melhorias, ajustes e necessidades do sistema.
Atuar no suporte às áreas Contábil, Fiscal, Administrativa e Financeira, garantindo o alinhamento entre os processos e o ERP.
Requisitos e qualificações
Cursando graduação em Sistemas de Informação, Ciências da Computação ou áreas relacionadas.
Conhecimento básico em sistemas ERP (preferencialmente Datasul).
Habilidade em comunicação para interação com usuários e fornecedores.
Proatividade, organização e vontade de aprender.
Leitura de manuais, documentação e materiais técnicos para suporte ao usuário e resolução de problemas
Entendimento do ciclo de vida e operação de sistemas computacionais.
Noções básicas de bancos de dados relacionais e não relacionais.
Conhecimentos básicos de consultas e manipulação de dados em SQL.
Habilidade em ferramentas como Word, Excel e PowerPoint."
7,4075025088,Faça sua Carreira de Engenharia de Processos no Itaú 🚀🧡,Itaú Unibanco,Híbrido,"São Paulo, SP",Há 2 semanas,,Assistente,Tempo integral,Bancos,"Vem construir no Itaú_
Como maior banco privado da América Latina, acompanhamos de perto o desenvolvimento de novas tecnologias e enxergamos o processo como uma evolução natural do nosso negócio, que
torna possível termos um contato ainda mais próximo com a experiência de nossos clientes
para continuarmos aprimorando suas jornadas.
Como é a área de Operações?
Ao longo de nossa história, desempenhamos um
papel fundamental na estratégia do Itaú Unibanco
. Dentro deste contexto, todos os produtos que oferecemos aos nossos clientes passam a ser produtos tecnológicos, independente da interface que ele escolha – seja ela física, digital ou mista. Vem construir soluções digitais em uma das maiores instituições financeiras da América Latina!
Como é a Carreira de Engenharia de Processos?
A Engenharia de processos no Itaú exerce um papel fundamental na busca constante da melhor jornada para nossos clientes, vem pra repensar o fundamental (ir na causa raiz das coisas,) buscar a reestruturação radical dos processos que visam alcançar mudanças disruptivas em indicadores críticos de desempenho, tais como qualidade, custos, riscos e agilidade. A pessoa engenheira de processos pode estar alocada em Squads, em comunidades ou nas áreas do banco atuando sempre em projetos de transformação e melhoria contínua (Build e Run).
Como será o seu dia a dia?
Garantir a aplicabilidade dos métodos para transformação das Jornadas fim a fim dos clientes.
Priorizar as ações quantificando o impacto através de uma Engenharia de Valor, garantindo assertividade nas escolhas que agregam.
Atuar no mapeamento de processos, resolução de problemas, construção e redesenho de jornadas, com foco no cliente, eficiência, qualidade e gestão de riscos.
Desdobrar as dores dos clientes e conectar com elementos de processo.
Realizar o estudo e a construção dos requisitos para os desenvolvimentos de tecnologia dos problemas assegurando o reuso de soluções.
Desenvolver soluções em No/Low-Code para transformação dos processos. Garante o teste funcional e homologação de requisitos de processos nas soluções desenvolvidas
Estrutura Gestão e Controle dos processos , aplicando ferramentas de qualidade , planejamento de demanda, intervenções operacionais, process mining , gestão da rotina , entre outras.
Modelo de trabalho: híbrido de 8x presenciais no mês
_Quais são as habilidades e competências necessárias?
Principais Habilidades Técnicas Com Experiência Prática
LEAN
Qualidade
PCP (Planejamento e Controle de Produção)
Gestão de Projetos/ Métodos Ágeis
Análise e exploração de Dados
Process Mining
Método de Solução de Problemas
Formação em Engenharia de Produção é um diferencial
_O que esperamos de você:
Habilidade em criticar de forma construtiva e provocativa;
Habilidade em negociação e construção de parcerias;
Capacidade analítica/ lógica
Habilidade em conectar conhecimento técnico com a estratégia dos projetos.
Organização, disciplina e pró-atividade
Habilidade em trabalhar em equipe de forma colaborativa,
Capacidade de tomada de decisões com base em processos e dados, foco no cliente e resultados;
🧡
"""

# Load Portuguese model
nlp = spacy.load("pt_core_news_md")

# Convert skill list into patterns
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp(skill) for skill in skill_list]
matcher.add("SKILLS", patterns)

# Process text
doc = nlp(sample_desc)

# Find matches
matches = matcher(doc)
skills_found = [doc[start:end].text for match_id, start, end in matches]

print("Extracted Skills:", set(skills_found))


Extracted Skills: {'Excel', 'PowerPoint', 'SQL'}


In [ ]:
import spacy
from spacy.matcher import PhraseMatcher
from thefuzz import process  # Fuzzy matching
import re

sample_desc = """Descrição
Estamos em busca de um(a) estagiário(a) para integrar nosso time de dados, focado na automatização e análise de dados, contribuindo diretamente para a tomada de decisão estratégica da empresa.
Responsabilidades e atribuições
Atuar na construção, governança e manutenção da arquitetura de dados do RPA.
Automatizar processo e otimização para reduzir esforços operacionais repetitivos.
Promover a cultura de dados dentro da empresa e junto aos clientes, auxiliando em projetos em andamento.
Participar da concessão, estruturação e organização de dados.
Contribuir para projetos de Inteligência Artificial (IA).
Apoiar a construção da arquitetura de dados e o desenvolvimento de novas funcionalidades que promovem escalabilidade e eficiência.
Requisitos e qualificações
Cursando graduação em Tecnologia, Engenharia, Estatística, Ciência de Dados ou áreas correlatas.
Noções de SQL para manipulação e extração de dados.
Familiaridade com ferramentas de BI (ex.: Metabase, powerbi).
Programação em Phyton e uso de bibliotecas para análise de dados (diferencial).
Conhecimento em ferramentas de RPA (diferencial).
Capacidade de levantamento e acompanhamento de KPIs em colaboração com usuários."""

# Load Portuguese NLP model
nlp = spacy.load("pt_core_news_md")

# Create PhraseMatcher for exact matches
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp(skill) for skill in skill_list]
matcher.add("SKILLS", patterns)

# Process text
doc = nlp(sample_desc)

# Extract exact matches using PhraseMatcher
matches = matcher(doc)
skills_found = [doc[start:end].text for match_id, start, end in matches]

# Extract all words from the text
words = set(re.findall(r"\b\w+\b", sample_desc))  # Get all unique words

# Apply fuzzy matching for words with a similarity score above a threshold (80%)
for word in words:
    best_match, score = process.extractOne(word, skill_list)
    if score > 93:  # Adjust threshold as needed
        skills_found.append(best_match)

# Remove duplicates
skills_found = set(skills_found)

print("Extracted Skills:", skills_found)


Extracted Skills: {'arquitetura de dados', 'SQL', 'análise de dados', 'Ciência de Dados', 'Inteligência Artificial', 'Estatística'}


In [79]:
process.extractOne('phyton', skill_list)

('Python', 83)